# 查询语句转换技术 -- 子查询分解

将复杂查询分解为更简单的子查询，以便更全面地检索信息。

将Query分解关键思想是将一个复杂问题分解成一系列更简单的子问题，然后依次解决它们。解决每个子问题都得益于之前解决的子问题的答案。


In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["MINIMAX_GROUP_ID"] = os.getenv("MINIMAX_GROUP_ID")
os.environ["MINIMAX_API_KEY"] = os.getenv("MINIMAX_API_KEY")

from langchain_community.chat_models import MiniMaxChat
from langchain.prompts import PromptTemplate

In [4]:
llm = MiniMaxChat(model="abab6.5s-chat")

subquery_decomposition_template = """
You are an AI assistant tasked with breaking down complex queries into simpler sub-queries for a RAG system.
Given the original query, decompose it into 2-4 simpler sub-queries that, when answered together, would provide a comprehensive response to the original query.

Original query: {original_query}

example: What are the impacts of climate change on the environment?

Sub-queries:
1. What are the impacts of climate change on biodiversity?
2. How does climate change affect the oceans?
3. What are the effects of climate change on agriculture?
4. What are the impacts of climate change on human health?

Note: If the original query is provided in Chinese, the sub-queries and their answers should be in Chinese. 
"""

subquery_decomposition_prompt = PromptTemplate(
    input_variables=["original_query"],
    template=subquery_decomposition_template
)

subquery_decomposer_chain = subquery_decomposition_prompt | llm

def decompose_query(original_query: str):
    response = subquery_decomposer_chain.invoke(original_query).content
    sub_queries = [q.strip() for q in response.split('\n') if q.strip() and not q.strip().startswith('Sub-queries:')]
    return sub_queries

In [5]:
original_query = "电信副卡能通过线上自助销户吗？"
sub_queries = decompose_query(original_query)
print("\nSub-queries:")
for i, sub_query in enumerate(sub_queries, 1):
    print(sub_query)


Sub-queries:
1. 电信副卡是什么，它与主卡有什么区别？
2. 电信副卡是否具备线上自助销户的功能？
3. 如何通过线上平台进行电信副卡的销户操作？
4. 电信副卡销户后，需要多久才能正式生效？
